## 什么是 RLHF（Reinforcement Learning from Human Feedback）

- **目标**：让模型更符合人类意图、更安全、更有用
- **核心思想**：
  - 用监督微调（SFT）教会模型基本的指令跟随
  - 用偏好数据训练奖励模型（RM），学会打分“更好/更差”的回答
  - 用强化学习（PPO）在奖励信号下优化策略，权衡质量、稳定性与多样性
- **关键组件**：指令数据、偏好数据（A/B 对比）、奖励模型、强化学习算法、KL 约束/参考策略
- **典型产物**：
  - SFT 模型（会做事）
  - RM 奖励模型（会打分）
  - PPO 后的对齐模型（做得更好）
  - DPO（取代 RM+PPO 的直接偏好优化）

### RLHF 的三阶段流程（工程化视角）

| 阶段 | 名称 | 作用 | 技术 |
|---|---|---|---|
| 1️⃣ | SFT（监督微调） | 教模型执行指令 | CrossEntropyLoss |
| 2️⃣ | 奖励模型（RM）训练 | 学会“什么样的回答更好” | Pairwise ranking (A > B) |
| 3️⃣ | PPO 强化优化 | 用奖励信号优化生成策略 | PPO 算法（Policy Gradient） |

### 1️⃣ SFT（监督微调）
- **输入**：指令-回答对（高质量、人类书写/筛选）
- **目标**：让模型基本学会“按指令作答”
- **训练**：最小化交叉熵损失（参考常用指令数据集）
- **输出**：SFT 模型（作为后续 RM/PPO 的参考策略）

### 2️⃣ 奖励模型（RM）训练
- **输入**：同一指令下成对回答（A、B），以及偏好标签（A > B）
- **目标**：学习“偏好评分函数” r(x, y)
- **训练**：Pairwise ranking（如 Bradley–Terry/Logistic loss）
- **输出**：能对任意回答打分的奖励模型

### 3️⃣ PPO 强化优化
- **输入**：SFT 模型作为初始策略 \(\pi_\theta\)，奖励模型 r 作为奖励信号
- **目标**：在 KL 约束下最大化期望奖励，提升对齐度与有用性
- **训练**：PPO（剪切策略梯度），引入 KL 惩罚以保持与参考策略接近
- **输出**：PPO 后的对齐模型（更符合人类偏好）
- **实践要点**：高质量偏好数据与稳定的 KL 控制是成功关键；监控长度偏置、模式坍缩与过拟合。

### DPO（Direct Preference Optimization）
- **定位**：作为第 3 阶段（PPO）的常见替代方案，用偏好对直接优化策略。
- **核心**：基于 \((x, y_{pos}, y_{neg})\) 提高 \(y_{pos}\) 概率、降低 \(y_{neg}\)，并以参考策略 \(\pi_{ref}\) 的对数概率差作隐式 KL 约束。
- **直观目标**：最小化 \(-\log \sigma\big(\beta[(\log \pi_\theta(y_{pos}|x) - \log \pi_\theta(y_{neg}|x)) - (\log \pi_{ref}(y_{pos}|x) - \log \pi_{ref}(y_{neg}|x))]\big)\)
- **优点**：流程简单、无奖励模型与 RL 回路、稳定易复现、吞吐高。
- **局限**：依赖高质量偏好数据；极端分布迁移下可控性较弱。

### 实验设置：模型与数据集选择
- **模型**：Qwen2.5-1.5B-Instruct（中文指令能力强，小参数、易于 LoRA/QLoRA）
- **SFT 数据**：BelleGroup/train_0.5M_CN（中文指令-回答对，体量适中，可采样）
- **偏好数据（用于 DPO/RM）**：argilla/ultrafeedback-binarized-preferences（成对偏好，易直接用于 DPO）

## 环境安装

In [14]:
%env TOKENIZERS_PARALLELISM=false
%pip install -q torch torchvision torchaudio
%pip install -q "transformers>=4.44.0" "datasets>=2.18.0" "accelerate>=0.33.0" \
                "peft>=0.12.0" "trl>=0.9.6" "sentencepiece>=0.1.99" "safetensors>=0.4.5" \
                "huggingface_hub>=0.24.0" "modelscope>=1.14.0" "protobuf>=4.25.0" \
                "numpy>=1.24.0" "scipy>=1.10.0" "tiktoken>=0.7.0"

# 仅在 CUDA 可用时安装 bitsandbytes（可选）
import torch
if torch.cuda.is_available():
    %pip install -q "bitsandbytes>=0.43.0"

# 打印关键版本，便于排查
import importlib.metadata as im
v = lambda n: (im.version(n) if n in {d.metadata['Name'] for d in im.distributions()} else 'N/A')
print("[Versions]",
      "torch=", v("torch"),
      "transformers=", v("transformers"),
      "datasets=", v("datasets"),
      "accelerate=", v("accelerate"),
      "peft=", v("peft"),
      "trl=", v("trl"),
      "modelscope=", v("modelscope"),
      "sentencepiece=", v("sentencepiece"),
      "bitsandbytes=", v("bitsandbytes"))


env: TOKENIZERS_PARALLELISM=false
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
[Versions] torch= 2.9.0 transformers= 4.57.1 datasets= 4.3.0 accelerate= 1.11.0 peft= 0.17.1 trl= 0.24.0 modelscope= 1.31.0 sentencepiece= 0.2.1 bitsandbytes= N/A


## 模型下载

In [ ]:
# 下载Qwen/Qwen2.5-1.5B-Instruct
import torch
from modelscope.hub.snapshot_download import snapshot_download
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "Qwen/Qwen2.5-1.5B-Instruct"  # ModelScope 上的模型标识（公共可直接下载）

# 选择设备
device = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")

# 通过 ModelScope 下载到本地缓存，然后用 Transformers 从本地目录加载
model_dir = snapshot_download(model_id)

tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_dir, trust_remote_code=True)
model = model.to(device)

print(f"[Device] {device}")

# 简单自检
txt = "你好，简要介绍一下你自己。"
inputs = tokenizer(txt, return_tensors="pt").to(device)
with torch.inference_mode():
    out = model.generate(**inputs, max_new_tokens=64, do_sample=False)
print(tokenizer.decode(out[0], skip_special_tokens=True))

2025-10-31 16:48:59,444 - modelscope - INFO - Target directory already exists, skipping creation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[Device] mps
你好，简要介绍一下你自己。 作为一个AI助手，我叫通义千问。我可以回答各种问题、提供信息和帮助您完成任务。有什么我能帮您的吗？


## 数据集下载

In [12]:
# 下载sft数据集和偏好数据集
from modelscope.hub.snapshot_download import snapshot_download

# 指定两个数据集名称（可按需修改）
sft_id = "AI-ModelScope/train_0.5M_CN"  # 中文数据集
pref_id_primary = "HuggingFaceH4/ultrafeedback_binarized" # 多语，英文为主，可筛出中文子集
pref_id_fallback = "AI-ModelScope/ultrafeedback-binarized-preferences-cleaned-kto" # UltraFeedback 兼容格式，亦可筛中文
 
# 仅使用 ModelScope 下载到本地缓存（不做回退）
sft_dir = snapshot_download(sft_id, repo_type="dataset")
pref_dir = snapshot_download(pref_id_primary, repo_type="dataset")
pref_dir_fallback = snapshot_download(pref_id_fallback, repo_type="dataset")

2025-10-31 17:25:39,264 - modelscope - INFO - Fetching dataset repo file list...


2025-10-31 17:25:40,635 - modelscope - INFO - Fetching dataset repo file list...


2025-10-31 17:25:42,623 - modelscope - INFO - Fetching dataset repo file list...


In [ ]:
# 加载预览数据
from datasets import load_dataset
import pandas as pd

pd.set_option("display.max_colwidth", None)

# 将嵌套结构规范为可读文本
def normalize_text(x):
    if x is None:
        return None
    if isinstance(x, str):
        return x
    if isinstance(x, dict):
        return x.get("content") or x.get("text") or x.get("value") or str(x)
    if isinstance(x, (list, tuple)):
        parts = []
        for it in x:
            if isinstance(it, str):
                parts.append(it)
            elif isinstance(it, dict):
                role = it.get("role")
                content = it.get("content") or it.get("text") or it.get("value")
                if content is not None:
                    parts.append((f"{role}: " if role else "") + str(content))
            else:
                parts.append(str(it))
        return "\n---\n".join(parts)
    return str(x)

# SFT：从本地目录加载并展示前 2 条（表格化，完整文本）
sft_preview = load_dataset(sft_dir, split="train[:2]")
sft_rows = []
for i in range(len(sft_preview)):
    ex = sft_preview[i]
    row = {
        "instruction": normalize_text(ex.get("instruction")),
        "input": normalize_text(ex.get("input")),
        "output": normalize_text(ex.get("output")),
        "instr_len": len(str(ex.get("instruction", ""))),
        "input_len": len(str(ex.get("input", ""))),
        "output_len": len(str(ex.get("output", ""))),
    }
    sft_rows.append(row)
print(f"[SFT] 预览 {len(sft_rows)} 条 / split=train[:2]")
display(pd.DataFrame(sft_rows))

# 偏好：优先主源，失败再用备用；split 兼容 train_prefs/train
try:
    pref_preview = load_dataset(pref_dir, split="train_prefs[:2]")
except Exception:
    try:
        pref_preview = load_dataset(pref_dir, split="train[:2]")
    except Exception:
        try:
            pref_preview = load_dataset(pref_dir_fallback, split="train_prefs[:2]")
        except Exception:
            pref_preview = load_dataset(pref_dir_fallback, split="train[:2]")

pref_rows = []
for i in range(len(pref_preview)):
    ex = pref_preview[i]
    prompt = ex.get("prompt") or ex.get("question") or ex.get("instruction") or ex.get("input")
    y_pos = ex.get("chosen") or ex.get("better_response") or ex.get("pos") or ex.get("preferred") or ex.get("y_pos")
    y_neg = ex.get("rejected") or ex.get("worse_response") or ex.get("neg") or ex.get("other") or ex.get("y_neg")

    prompt = normalize_text(prompt)
    y_pos = normalize_text(y_pos)
    y_neg = normalize_text(y_neg)

    row = {
        "prompt": prompt,
        "y_pos": y_pos,
        "y_neg": y_neg,
        "prompt_len": len(str(prompt or "")),
        "y_pos_len": len(str(y_pos or "")),
        "y_neg_len": len(str(y_neg or "")),
    }
    pref_rows.append(row)
print(f"[Preference] 预览 {len(pref_rows)} 条（完整文本显示）")
display(pd.DataFrame(pref_rows))


[SFT] 预览 2 条 / split=train[:2]


,instruction,input,output,instr_len,input_len,output_len
0,给定一个英文句子，翻译成中文。\nI love to learn new things every day.\n,,我每天喜欢学习新事物。,54,0,11
1,给定一个文字输入，将其中的所有数字加1。\n“明天的会议在9点开始，记得准时到达。”\n,,“明天的会议在10点开始，记得准时到达。”,42,0,21


[Preference] 预览 2 条（完整文本显示）


,prompt,y_pos,y_neg,prompt_len,y_pos_len,y_neg_len
0,how can i develop a habit of drawing daily,"user: how can i develop a habit of drawing daily\n---\nassistant: Developing a daily habit of drawing can be challenging but with consistent practice and a few tips, it can become an enjoyable and rewarding part of your daily routine. Here are some strategies to help you develop the habit of drawing daily:\n\n1. Set a specific time: Allocate a specific time of the day to draw. It could be in the morning, afternoon, or evening. Make drawing a part of your daily routine.\n2. Set a specific duration: Determine the amount of time you want to spend on drawing each day. It can be as little as 10 minutes or as long as an hour. Be consistent with the duration to help build the habit.\n3. Start small and simple: Don't try to create a masterpiece every day, start with simple and easy-to-do sketches. Focus on improving your skills gradually.\n4. Use a variety of tools and mediums: Experiment with different tools like pencils, pens, markers, and different mediums like paper, canvas, or digital apps to keep your drawing practice interesting and engaging.\n5. Take breaks and rest: Taking breaks and resting after some time of drawing can help you avoid burnout and stay motivated.\n6. Challenge yourself: Set challenges like drawing objects from memory or a specific subject to improve your skills and keep your drawing practice interesting.\n7. Track your progress: Keep a record of your daily drawing practice and track your progress. This can be a source of motivation and help you see how far you've come.\n\nRemember, developing a habit takes time and patience. Stay consistent with your drawing practice, be flexible and open to trying new things, and with time, you'll develop a habit of daily drawing that brings you joy and satisfaction.","user: how can i develop a habit of drawing daily\n---\nassistant: As an AI language model, I cannot personally develop habits for you. But, here are some tips for developing a habit of drawing daily:\n\n1. Start small: Start with simple drawings or doodles and gradually increase the complexity of your drawings.\n\n2. Set a schedule: Set a specific time every day to draw, and try to stick to it as much as possible.\n\n3. Make it fun: Don't pressure yourself to create masterpieces every time you draw. Make it a relaxing and enjoyable experience.\n\n4. Use resources: There are many drawing tutorials available online. Use resources like YouTube or online drawing courses to help you improve your skills.\n\n5. Surround yourself with inspiration: Expose yourself to a variety of art forms, such as paintings, illustrations, and photographs, to inspire and motivate you.\n\nRemember, everyone has their own creative style and pace. Just keep practicing and enjoying the process of drawing.",42,1737,977
1,how can I transform the getPosition method of antv/g's group in zrender?,"user: how can I transform the getPosition method of antv/g's group in zrender?\n---\nassistant: It is not recommended to modify built-in methods as it can lead to unexpected results and potential bugs. You may consider developing a new method or exploring other methods to achieve your desired outcome. Alternatively, you can search for other libraries or modules that offer similar functionalities or reach out to the library's support team for assistance.","user: how can I transform the getPosition method of antv/g's group in zrender?\n---\nassistant: Thank you for reaching out for assistance! I'm here to help you with your question. However, I must point out that the question itself may not be meaningful.\n\nThe `getPosition` method is a part of the AntV/G library, which is a gradient boosting framework. It is not clear what you mean by ""transforming"" this method, as it is not a functional programming concept. Additionally, the concept of ""zrender"" is not related to AntV/G.\n\nCould you please provide more context or clarify your question? I'd be happ